# Setup

Import Semantic Kernel SDK from pypi.org

In [35]:
%pip install -U semantic-kernel --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
from semantic_kernel import __version__

__version__

'1.22.0'

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

>NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.

#### Using Azure OpenAI

Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to the `.env` file in the same folder:

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The names should match the names used in the `.env` file, as shown above.

For more advanced configuration, please follow the steps outlined in the [setup guide](./CONFIGURING_THE_KERNEL.md).

Let's define our kernel for this example.

In [6]:
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)

We now configure our Chat Completion service on the kernel.

In [17]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    service_id="default"
)

Let's try chat with LLM model.

In [18]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory

execution_settings = AzureChatPromptExecutionSettings()

system_message = """
You are a helpful assistant that can answer questions and provide information.
"""

chat_history = ChatHistory(system_message=system_message)

chat_history.add_user_message("Hello, how are you?")

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

print(response)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [20]:
from semantic_kernel import Kernel

kernel = Kernel()

kernel.add_service(chat_completion)



In [21]:
from typing import Annotated
from semantic_kernel.functions import kernel_function

class LightsPlugin:
    lights = [
        {"id": 1, "name": "Table Lamp", "is_on": False},
        {"id": 2, "name": "Porch light", "is_on": False},
        {"id": 3, "name": "Chandelier", "is_on": True},
    ]

    @kernel_function(
        name="get_lights",
        description="Gets a list of lights and their current state",
    )
    def get_state(
        self,
    ) -> str:
        """Gets a list of lights and their current state."""
        return self.lights

    @kernel_function(
        name="change_state",
        description="Changes the state of the light",
    )
    def change_state(
        self,
        id: int,
        is_on: bool,
    ) -> str:
        """Changes the state of the light."""
        for light in self.lights:
            if light["id"] == id:
                light["is_on"] = is_on
                return light
        return None

In [ ]:
from semantic_kernel.utils.logging import setup_logging
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory
import logging

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

# Set the logging level for  semantic_kernel.kernel to DEBUG.
setup_logging()
logging.getLogger("kernel").setLevel(logging.DEBUG)

# Add a plugin (the LightsPlugin class is defined below)
kernel.add_plugin(
    LightsPlugin(),
    plugin_name="Lights",
)
# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
# Create a history of the conversation
history = ChatHistory()
# Initiate a back-and-forth chat
userInput = None
while True:
    # Collect user input
    userInput = input("User > ")
    # Terminate the loop if the user says "exit"
    if userInput == "exit":
        break
    # Add user input to the history
    history.add_user_message(userInput)
    # Get the response from the AI
    result = await chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )
    # Print the results
    print("Assistant > " + str(result))
    # Add the message from the agent to the chat history
    history.add_message(result)

# Run a Semantic Function

**Step 3**: Load a Plugin and run a semantic function:


In [6]:
plugin = kernel.add_plugin(parent_directory="./", plugin_name="FunPlugin")

In [7]:
from semantic_kernel.functions import KernelArguments

joke_function = plugin["Joke"]

joke = await kernel.invoke(
    joke_function,
    KernelArguments(input="time travel to dinosaur age", style="super silly"),
)
print(joke)

Function failed. Error: Error occurred while invoking function Joke: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", NotFoundError("Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}"))
Something went wrong in function invocation. During function invocation: 'FunPlugin-Joke'. Error description: 'Error occurred while invoking function Joke: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", NotFoundError("Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}"))'


KernelInvokeException: Error occurred while invoking function: 'FunPlugin-Joke'